In [1]:
import pandas as pd
from pam.read import read_matsim
from pam.planner import choice
from pam.operations.cropping import link_population
from pam.planner.od import OD, Labels
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
population = read_matsim(os.path.join('..', 'tests', 'test_data', 'test_matsim_plansv12.xml'))
link_population(population)
for hid, pid, person in population.people():
    for act in person.activities:
        act.location.area = 'a' 

In [4]:
data_zones = pd.DataFrame(
    {
        'zone': ['a', 'b'],
        'jobs': [100, 200],
        'schools': [3, 1]
    }
).set_index('zone')
data_zones

,jobs,schools
zone,,
a,100,3
b,200,1


In [148]:
data_od = np.array(
    [
        [
            [[20, 40], [40, 20]],
            [[5, 8], [8, 5]]
        ],
        [
            [[30, 45], [45, 30]],
            [[5, 9], [10, 5]]
        ]
    ]
)

labels = {
    'mode': ['car', 'bus'],
    'vars': ['time', 'distance'],
    'origin_zones': ['a', 'b'],
    'destination_zones': ['a', 'b']
}


od = OD(
    data=data_od,
    labels=labels
)

od

Origin-destination dataset 
--------------------------------------------------
Labels(vars=['time', 'distance'], origin_zones=['a', 'b'], destination_zones=['a', 'b'], mode=['car', 'bus'])
--------------------------------------------------
time - car:
[[20 40]
 [ 5  8]]
--------------------------------------------------
time - bus:
[[40 20]
 [ 8  5]]
--------------------------------------------------
distance - car:
[[30 45]
 [ 5 10]]
--------------------------------------------------
distance - bus:
[[45 30]
 [ 9  5]]
--------------------------------------------------

In [149]:
planner = choice.ChoiceMNL(population, od, data_zones)

In [277]:
scope = "act.act=='work'"
asc = [0, 1]
# asc = 10
beta_time = [1, 2] # one beta for each mode
u = f""" \
    {asc} + (1 * (person.attributes['subpopulation']=='poor')) * \
    (zones['jobs'].values / ({beta_time} * od['time', person.home.area]))
"""
# u = f"""od['distance', 'b']"""

choice_set = planner.get_choice_set(u, scope)
choice_set

ChoiceSet(idxs=[{'pid': 'chris', 'hid': 'chris', 'seq': 1, 'act': <pam.activity.Activity object at 0x7f56ababd4c0>}, {'pid': 'fatema', 'hid': 'fatema', 'seq': 1, 'act': <pam.activity.Activity object at 0x7f56ababd7c0>}, {'pid': 'fred', 'hid': 'fred', 'seq': 3, 'act': <pam.activity.Activity object at 0x7f56ababf250>}, {'pid': 'gerry', 'hid': 'gerry', 'seq': 3, 'act': <pam.activity.Activity object at 0x7f56ababfe50>}, {'pid': 'nick', 'hid': 'nick', 'seq': 1, 'act': <pam.activity.Activity object at 0x7f56abac6070>}], u_choices=array([[0. , 1. , 0. , 1. ],
       [5. , 3.5, 2.5, 6. ],
       [5. , 3.5, 2.5, 6. ],
       [5. , 3.5, 2.5, 6. ],
       [0. , 1. , 0. , 1. ]]), choice_labels=[('a', 'car'), ('a', 'bus'), ('b', 'car'), ('b', 'bus')])

Select:

In [279]:
selections = planner.get_selections(u, scope)
selections.selections

[('a', 'bus'), ('a', 'bus'), ('b', 'bus'), ('a', 'car'), ('b', 'bus')]

Apply new locations and mode:

In [274]:
planner.apply(u, scope)